In [1]:
%load_ext autoreload
%autoreload 2

# Automated formant measurements

This notebook illustrates how to run several formant trackers on a corpus of audio files and collect the results in order to do ensemble analysis. The ensemble technique involves generating formant estimates using several different algorithms, then choosing the best fit.

The X-Ray Microbeam database will be used for the illustration. This database contains audio and associated articulatory data from 48 speakers. Aligned word and phone labels for the audio are available separately.

In this notebook we generate and cache three formant estimates with (1) the inverse filter control method using `ifcformant` and via LPC analysis with the ESPS `formant` command's (2) covariance and (3) autocorrelation modes. 

There are three sources of background information that will make it easier to follow this notebook, and it is recommended that you consult them if you are lost are want more detail.

1. The [Post-processing and data collection notebook](https://github.com/rsprouse/phonlab/blob/master/doc/post-processing_and_data_collection.ipynb) illustrates the post-processing workflow employed in this notebook.
2. The [`dir2df()` notebook](https://github.com/rsprouse/phonlab/blob/master/doc/Retrieving%20filenames%20in%20a%20directory%20tree%20with%20%60dir2df%28%29%60.ipynb) goes into more detail on the `dir2df()` function, which is used in the post-processing notebook.
1. The [X-Ray Microbeam Database repo](https://github.com/rsprouse/xray_microbeam_database) provides the annotations to the X-Ray Microbeam Database and links to the original data files.

## The workflow

The general workflow for processing files is as follows:

1. Read the input data filenames (*.wav) into a DataFrame.
1. Mirror the source directory structure in the destination directory.
1. Merge input and output filenames to find input files that don't have a corresponding output file.
1. Read existing post-processed formant output filenames, if any, into a DataFrame.
1. Run the formant analysis for each input file that is missing an output and cache the results.

The steps will be illustrated separately for each of the formant analysis methods.       

In [1]:
import os, re
import subprocess
import pandas as pd
from phonlab.utils import dir2df
from temputils import fb2df

First we define file locations. The name of the directory containing the microbeam database files is stored in `xraydir`, and three additional cache directories names are stored in `ifcdir`, `covdir`, and `acdir`. These cache directories are where output files for each of the analysis methods will be saved.

In [2]:
xraydir = '/media/sf_Downloads/xray_microbeam_database'
ifcdir = '/media/sf_Downloads/xray_formants_ifc'  # For ifcformant output files
covdir = '/media/sf_Downloads/xray_formants_cov'  # For covariance method output files
acdir = '/media/sf_Downloads/xray_formants_ac'    # For autocorrelation method output files
#rbdir = '/Users/ronald/Downloads/xray_formants_rb'    # For robust lpc method output files
#analysis = pd.DataFrame.from_records([
#    {'method': 'ifc', 'dir': '/Users/ronald/Downloads/xray_formants_ifc', 'func': run_ifcformant},
#    {'method': 'cov', 'dir': '/Users/ronald/Downloads/xray_formants_cov', 'func': run_formant_cov},
#    {'method': 'ac', 'dir': '/Users/ronald/Downloads/xray_formants_ac', 'func': run_formant_ac},
#    {'method': 'rb', 'dir': '/Users/ronald/Downloads/xray_formants_rb', 'func': run_robustlpc},
#])
#analysis

## Load speaker metadata

Speaker metadata for the xray database is found in the 'speaker_metadata.csv' file. Use the `read_csv()` function to read the 'subject' and 'sex' columns from it. The .csv file uses 'F' and 'M' in the 'sex' column, and the `converters` parameter replaces these values with 'female' and 'male', which are values compatible with `ifcformant`. 

In [3]:
md = pd.read_csv(
    os.path.join(xraydir, 'speaker_metadata.csv'),
    usecols=['subject', 'sex'],
    converters={'sex': lambda x: 'female' if x == 'F' else 'male'}
)
md.head()

FileNotFoundError: File b'/media/sf_Downloads/xray_microbeam_database/speaker_metadata.csv' does not exist

Add lpc analysis parameters based on sex. The values are defined as strings since they will be used for command line arguments rather than for calculations or comparisons.

In [5]:
lpcargs = pd.DataFrame.from_records([
    ('female', '12', '600'),
    ('male',   '14', '500'),
    ('child',  '10', '700')
], columns=['sex', 'lpc_order', 'nom_f1'])
lpcargs

,sex,lpc_order,nom_f1
0,female,12,600
1,male,14,500
2,child,10,700


In [6]:
md = md.merge(lpcargs, on='sex', how='left')
md.head()

,subject,sex,lpc_order,nom_f1
0,JW05,female,12,600
1,JW06,male,14,500
2,JW07,male,14,500
3,JW08,male,14,500
4,JW09,female,12,600


# Load xray filenames

Load the .wav files in the xray database.

In [7]:
wavdf = dir2df(
    xraydir,
    dirpat='^JW',
    fnpat='\.wav$',
#    fnpat='^tp(?P<task>\d+)\.wav$',  # also include ta*.wav?
    addcols=['dirname', 'barename']
)
wavdf.head()

,dirname,relpath,fname,barename
0,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta001.wav,ta001
1,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta002.wav,ta002
2,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta003.wav,ta003
3,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta004.wav,ta004
4,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta005.wav,ta005


## Ensure destination directories are created

Output files will be written to a directory structure that is like the one found in the xray data directory. The next cell creates the subdirectories of `xraydir` in `formantdir` if they don't already exist, in preparation for writing formant analysis output files.

The `os.makedirs()` function automatically creates intermediate directories if necessary for nested subdirectories. Setting the `exist_ok` parameter to `True` makes it safe to use this function without checking whether any of the destination directories already exist (by default `os.makedirs()` will throw an `OSError` if you attempt to re-create an existing directory), which means you can safely use the following loop to keep a destination directory in sync incrementally with a data directory as more data is added.

In [8]:
for d in wavdf.relpath.unique():
    os.makedirs(os.path.join(ifcdir, d), exist_ok=True)

## Merge with speaker metadata.

In [9]:
wavdf = wavdf.merge(
    md,
    left_on='relpath',
    right_on='subject',
    how='left'
)
wavdf

,dirname,relpath,fname,barename,subject,sex,lpc_order,nom_f1
0,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta001.wav,ta001,JW11,male,14,500
1,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta002.wav,ta002,JW11,male,14,500
2,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta003.wav,ta003,JW11,male,14,500
3,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta004.wav,ta004,JW11,male,14,500
4,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta005.wav,ta005,JW11,male,14,500
5,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta006.wav,ta006,JW11,male,14,500
6,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta007.wav,ta007,JW11,male,14,500
7,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta008.wav,ta008,JW11,male,14,500
8,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta009.wav,ta009,JW11,male,14,500
9,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta010.wav,ta010,JW11,male,14,500


## Add expected file extensions

We expect to find '.ifc', '.fbc', and '.fba' output files for every input '.wav' file. We create columns for each of these extensions that will be used as merge keys when merging with existing output files.

In [10]:
wavdf = wavdf.assign(ifcext='.ifc')
wavdf

,dirname,relpath,fname,barename,subject,sex,lpc_order,nom_f1,ifcext
0,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta001.wav,ta001,JW11,male,14,500,.ifc
1,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta002.wav,ta002,JW11,male,14,500,.ifc
2,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta003.wav,ta003,JW11,male,14,500,.ifc
3,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta004.wav,ta004,JW11,male,14,500,.ifc
4,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta005.wav,ta005,JW11,male,14,500,.ifc
5,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta006.wav,ta006,JW11,male,14,500,.ifc
6,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta007.wav,ta007,JW11,male,14,500,.ifc
7,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta008.wav,ta008,JW11,male,14,500,.ifc
8,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta009.wav,ta009,JW11,male,14,500,.ifc
9,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta010.wav,ta010,JW11,male,14,500,.ifc


# Load existing formant output files into a DataFrame

In [11]:
ifcdf = dir2df(ifcdir, addcols=['barename', 'dirname', 'ext'])
#covdf = dir2df(covdir, addcols=['barename', 'dirname', 'ext'])
#acdf = dir2df(acdir, addcols=['barename', 'dirname', 'ext'])
ifcdf

,dirname,relpath,fname,barename,ext


## Merge with data directory

In [12]:
ifcdf = wavdf.merge(
    ifcdf,
    left_on=['relpath', 'barename', 'ifcext'],
    right_on=['relpath', 'barename', 'ext'],
    suffixes=['', '_ifc'],
    how='left'
)
#covdf = wavdf.merge(
#    covdf,
#    left_on=['relpath', 'barename', 'covext'],
#    right_on=['relpath', 'barename', 'ext'],
#    suffixes=['', '_cov'],
#    how='left'
#)
#acdf = wavdf.merge(
#    acdf,
#    left_on=['relpath', 'barename', 'acext'],
#    right_on=['relpath', 'barename', 'ext'],
#    suffixes=['', '_ac'],
#    how='left'
#)
ifcdf

,dirname,relpath,fname,barename,subject,sex,lpc_order,nom_f1,ifcext,dirname_ifc,fname_ifc,ext
0,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta001.wav,ta001,JW11,male,14,500,.ifc,NaN,NaN,NaN
1,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta002.wav,ta002,JW11,male,14,500,.ifc,NaN,NaN,NaN
2,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta003.wav,ta003,JW11,male,14,500,.ifc,NaN,NaN,NaN
3,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta004.wav,ta004,JW11,male,14,500,.ifc,NaN,NaN,NaN
4,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta005.wav,ta005,JW11,male,14,500,.ifc,NaN,NaN,NaN
5,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta006.wav,ta006,JW11,male,14,500,.ifc,NaN,NaN,NaN
6,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta007.wav,ta007,JW11,male,14,500,.ifc,NaN,NaN,NaN
7,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta008.wav,ta008,JW11,male,14,500,.ifc,NaN,NaN,NaN
8,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta009.wav,ta009,JW11,male,14,500,.ifc,NaN,NaN,NaN
9,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta010.wav,ta010,JW11,male,14,500,.ifc,NaN,NaN,NaN


## Find missing formant output files

We expect to find an '.ifc' file for every '.wav' file in the data directory. The preceding merge inserts a NaN value into the `fname_ifc` column wherever a '.wav' file does not have a corresponding '.ifc' file. 

In [13]:
missingifc = ifcdf[ifcdf.fname_ifc.isna()]
missingifc.dirname_ifc = ifcdir
missingifc.fname_ifc = missingifc.barename + missingifc.ifcext

#missingcov = covdf[covdf.fname_cov.isna()]
#missingcov.dirname_cov = covdir
#missingcov.fname_cov = missingcov.barename + missingcov.covext

#missingac = acdf[acdf.fname_ac.isna()]
#missingac.dirname_ac = acdir
#missingac.fname_ac = missingac.barename + missingac.acext

missingifc

,dirname,relpath,fname,barename,subject,sex,lpc_order,nom_f1,ifcext,dirname_ifc,fname_ifc,ext
0,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta001.wav,ta001,JW11,male,14,500,.ifc,/Users/ronald/Downloads/xray_formants_ifc,ta001.ifc,NaN
1,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta002.wav,ta002,JW11,male,14,500,.ifc,/Users/ronald/Downloads/xray_formants_ifc,ta002.ifc,NaN
2,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta003.wav,ta003,JW11,male,14,500,.ifc,/Users/ronald/Downloads/xray_formants_ifc,ta003.ifc,NaN
3,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta004.wav,ta004,JW11,male,14,500,.ifc,/Users/ronald/Downloads/xray_formants_ifc,ta004.ifc,NaN
4,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta005.wav,ta005,JW11,male,14,500,.ifc,/Users/ronald/Downloads/xray_formants_ifc,ta005.ifc,NaN
5,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta006.wav,ta006,JW11,male,14,500,.ifc,/Users/ronald/Downloads/xray_formants_ifc,ta006.ifc,NaN
6,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta007.wav,ta007,JW11,male,14,500,.ifc,/Users/ronald/Downloads/xray_formants_ifc,ta007.ifc,NaN
7,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta008.wav,ta008,JW11,male,14,500,.ifc,/Users/ronald/Downloads/xray_formants_ifc,ta008.ifc,NaN
8,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta009.wav,ta009,JW11,male,14,500,.ifc,/Users/ronald/Downloads/xray_formants_ifc,ta009.ifc,NaN
9,/Users/ronald/Downloads/xray_microbeam_database,JW11,ta010.wav,ta010,JW11,male,14,500,.ifc,/Users/ronald/Downloads/xray_formants_ifc,ta010.ifc,NaN


## Define analysis functions


In [14]:
def run_ifcformant(row, errors='raise'):
    '''Run ifcformant.'''
    try:
        subprocess.check_call([
            "ifcformant",
            "--speaker", row.sex,
            "--print-header",
            "--output", os.path.join(row.dirname_ifc, row.relpath, row.fname_ifc),
            os.path.join(row.dirname, row.relpath, row.fname)
        ])
    except subprocess.CalledProcessError as e:
        if errors == 'ignore':
            pass
        else:
            raise e

def run_formant_cov(row, errors='raise'):
    '''Run ESPS formant command with covariance settings.'''
    wdur = "0.025"
    try:
        subprocess.check_call([
            "formant",
            "-o", row.lpc_order,
            "-N", row.nom_f1,
            "-t1",
            "-w", wdur,
            "-O", os.path.join(row.dirname_cov, row.relpath),
            os.path.join(row.dirname, row.relpath, row.fname)
        ])
    except subprocess.CalledProcessError as e:
        if errors == 'ignore':
            pass
        else:
            raise e

def run_formant_ac(row, errors='raise'):
    '''Run ESPS formant command with autocorrelation settings.'''
    wdur = "0.049"
    subprocess.run(["formant","-o",str(order), "-N",str(nom_f1), "-t0","-w",str(wdur), soundfile],cwd=root)
    try:
        subprocess.check_call([
            "formant",
            "-o", row.lpc_order,
            "-N", row.nom_f1,
            "-t0",
            "-w", wdur,
            "-O", os.path.join(row.dirname_ac, row.relpath),
            os.path.join(row.dirname, row.relpath, row.fname)
        ])
    except subprocess.CalledProcessError as e:
        if errors == 'ignore':
            pass
        else:
            raise e


In [314]:
#index=0.10*np.arange(len(d)), 
f = EspsFea('/Users/ronald/Downloads/xray_formants_ac/syba_di.fb')
d = f.get_data()
pd.DataFrame.from_records(d.tolist(), columns=['f1', 'f2', 'f3', 'f4', 'bw1', 'bw2', 'bw3', 'bw4'])

8

In [16]:
fbdf = fb2df('/Users/ronald/Downloads/xray_formants_ac/syba_di.fb', step=0.010)
fbdf

,t1,f1,f2,f3,f4,bw1,bw2,bw3,bw4
0,0.00,748.775536,1676.143784,3271.790973,3993.814557,200.715158,541.653987,231.539310,451.179899
1,0.01,1325.861552,3499.634595,3561.345489,3499.634595,464.116803,1175.167209,599.266338,1175.167209
2,0.02,1320.114658,2130.588354,3561.345489,4719.852502,509.196019,722.811535,599.266338,651.670560
3,0.03,216.069013,1558.454221,2833.607805,3770.130892,333.463417,594.334566,613.061859,411.481336
4,0.04,66.481432,1388.135893,3137.876954,3906.246691,407.456662,378.132720,568.367027,659.455810
5,0.05,1389.779935,2160.531302,3109.417449,3996.506962,1697.052156,691.212223,797.234775,478.799744
6,0.06,1086.079082,2135.775982,2891.133427,3889.785375,885.021126,453.480333,552.336940,413.152531
7,0.07,1096.585509,2326.764951,3633.013769,4399.733808,887.141971,429.924268,451.666941,255.183819
8,0.08,1005.039576,2252.792991,3099.128189,4345.855490,892.054812,322.679756,750.529722,245.666225
9,0.09,241.780083,2275.134593,3796.830547,4323.858983,488.996261,663.437109,569.768945,228.018557
